# Chatbot Tutor with RAG System

This notebook implements a complete chatbot tutor that uses Retrieval-Augmented Generation (RAG) to answer questions based on uploaded PDF documents. The system creates a local vector index for efficient information retrieval.

## 1. Install Required Libraries

In [24]:
!pip install -q langchain langchain-community langchain-openai langchain-text-splitters
!pip install -q chromadb
!pip install -q pypdf
!pip install -q sentence-transformers
!pip install -q openai
!pip install -q tiktoken
!pip install -q sounddevice soundfile
!pip install -q sounddevice
!pip install -q gTTS
!apt-get install -y portaudio19-dev

print("✓ All packages installed successfully!")

Reading package lists... Done
Reading package lists... Done0%
Building dependency tree... Done
Reading state information... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libasound2-dev libjack-dev libjack0 libportaudio2 libportaudiocpp0
Suggested packages:
  libasound2-doc jackd1 portaudio19-doc
The following packages will be REMOVED:
  libjack-jackd2-0
The following NEW packages will be installed:
  libasound2-dev libjack-dev libjack0 libportaudio2 libportaudiocpp0
The following additional packages will be installed:
  libasound2-dev libjack-dev libjack0 libportaudio2 libportaudiocpp0
Suggested packages:
  libasound2-doc jackd1 portaudio19-doc
The following packages will be REMOVED:
  libjack-jackd2-0
The following NEW packages will be installed:
  libasound2-dev libjack-dev libjack0 libportaudio2 libportaudiocpp0
  portaudio19-dev
0 upgraded, 6 newly installed, 1 to remove and 41 not upgraded.
Need to ge

In [25]:
# Check installed langchain packages
import subprocess
result = subprocess.run(['pip', 'list'], capture_output=True, text=True)
langchain_packages = [line for line in result.stdout.split('\n') if 'langchain' in line.lower()]
for pkg in langchain_packages:
    print(pkg)

langchain                                1.1.0
langchain-classic                        1.0.0
langchain-community                      0.4.1
langchain-core                           1.1.0
langchain-openai                         1.1.0
langchain-text-splitters                 1.0.0


## 2. Import Required Libraries

In [26]:
import os
import tempfile
import warnings
from pathlib import Path
from typing import Optional

# Audio + display helpers
try:
    import sounddevice as sd
except OSError as exc:
    sd = None
    print(
        "sounddevice/PortAudio unavailable ("
        + str(exc)
        + ") — microphone recording disabled. Install system PortAudio libraries to enable voice capture.",
    )
except ImportError:
    sd = None
    print("sounddevice not installed — run `pip install sounddevice` to enable voice capture.")

import soundfile as sf
from IPython.display import Audio, display

try:
    from gtts import gTTS
except ImportError:
    gTTS = None
    print("gTTS not installed — run `pip install gTTS` for the fallback text-to-speech path.")

# LangChain imports
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain_classic.chains import ConversationalRetrievalChain
from langchain_classic.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate

from openai import OpenAI

warnings.filterwarnings('ignore')

## 3. Configuration Settings

In [27]:
# Configuration
OPENROUTER_API_KEY = "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"  # Your OpenRouter API key
MODEL_NAME = "openai/gpt-4o-mini"  # OpenRouter model name
PERSIST_DIRECTORY = "./chroma_db"
PDF_DIRECTORY = "./uploaded_pdfs"

# Audio / voice defaults
VOICE_SAMPLE_RATE = 16000
VOICE_RECORD_SECONDS = 10
STT_MODEL = "openai/whisper-1"
TTS_MODEL = "openai/gpt-4o-mini-tts"
TTS_VOICE = "alloy"

# Create directories if they don't exist
os.makedirs(PERSIST_DIRECTORY, exist_ok=True)
os.makedirs(PDF_DIRECTORY, exist_ok=True)

# Set API key and base URL for OpenRouter
os.environ["OPENAI_API_KEY"] = OPENROUTER_API_KEY
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

print("✓ Configuration completed")
print(f"✓ Using OpenRouter with model: {MODEL_NAME}")
print(f"✓ Vector database directory: {PERSIST_DIRECTORY}")
print(f"✓ PDF upload directory: {PDF_DIRECTORY}")
print(f"✓ STT model: {STT_MODEL} | TTS model: {TTS_MODEL} ({TTS_VOICE})")
if os.getenv("OPENAI_TTS_API_KEY") or "api.openai.com" in os.getenv("OPENAI_API_BASE", "").lower():
    print("✓ Cloud TTS credentials detected — GPT-4o TTS requests will use them when available.")
else:
    print("⚠ No cloud TTS credentials detected; TTS will fall back to the local gTTS helper.")

✓ Configuration completed
✓ Using OpenRouter with model: openai/gpt-4o-mini
✓ Vector database directory: ./chroma_db
✓ PDF upload directory: ./uploaded_pdfs
✓ STT model: openai/whisper-1 | TTS model: openai/gpt-4o-mini-tts (alloy)


### Optional: Voice Controls (Speech-to-Text + Text-to-Speech)
- Requires the extra packages installed above plus microphone access (local runtime) or manual audio uploads.
- If PortAudio drivers are missing you can still use `voice-file <path>` to transcribe recorded clips; install system PortAudio libraries to unlock live recording.
- Whisper STT requests are routed through OpenRouter. TTS first tries GPT-4o Mini TTS when `OPENAI_TTS_API_KEY` (or an OpenAI base URL) is provided and gracefully falls back to the local `gTTS` helper otherwise.
- Voice mode integrates with the chat loop via simple commands (details near the interface section).

In [28]:
def _get_openrouter_client() -> OpenAI:
    api_key = os.getenv("OPENAI_API_KEY")
    base_url = os.getenv("OPENAI_API_BASE", "https://openrouter.ai/api/v1")
    if not api_key:
        raise EnvironmentError("OPENAI_API_KEY is missing. Export it before using voice features.")
    return OpenAI(base_url=base_url, api_key=api_key)


def _get_tts_client() -> Optional[OpenAI]:
    """Return a client configured for high-quality TTS if credentials are available."""
    tts_key = os.getenv("OPENAI_TTS_API_KEY")
    tts_base = os.getenv("OPENAI_TTS_API_BASE", "https://api.openai.com/v1")
    if tts_key:
        return OpenAI(base_url=tts_base, api_key=tts_key)
    api_base = os.getenv("OPENAI_API_BASE", "").lower()
    api_key = os.getenv("OPENAI_API_KEY")
    if api_key and "api.openai.com" in api_base:
        return OpenAI(base_url=os.getenv("OPENAI_API_BASE"), api_key=api_key)
    return None


def record_audio_to_file(duration: int = VOICE_RECORD_SECONDS, samplerate: int = VOICE_SAMPLE_RATE) -> Path:
    """Record audio from the default microphone into a temporary WAV file."""
    if sd is None:
        raise RuntimeError(
            "sounddevice/PortAudio is unavailable in this environment. Use `voice-file <path>` instead or install PortAudio libs.",
        )
    temp_path = Path(tempfile.mkstemp(suffix=".wav", prefix="tutor_question_")[1])
    try:
        frames = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=1, dtype="float32")
        sd.wait()
        sf.write(str(temp_path), frames, samplerate)
    except Exception as exc:
        raise RuntimeError(f"Audio recording failed: {exc}")
    return temp_path


def transcribe_audio_file(audio_path: Path, model: str = STT_MODEL) -> str:
    client = _get_openrouter_client()
    with open(audio_path, "rb") as audio_file:
        transcript = client.audio.transcriptions.create(
            model=model,
            file=audio_file,
        )
    return transcript.text.strip()


def capture_and_transcribe_question(duration: int = VOICE_RECORD_SECONDS) -> str:
    audio_path = record_audio_to_file(duration=duration)
    try:
        text = transcribe_audio_file(audio_path)
    finally:
        try:
            audio_path.unlink(missing_ok=True)
        except Exception:
            pass
    return text


def synthesize_speech_to_file(text: str, model: str = TTS_MODEL, voice: str = TTS_VOICE) -> Path:
    """Synthesize speech via OpenAI/OpenRouter when possible, fallback to gTTS otherwise."""
    tts_client = _get_tts_client()
    audio_path = Path(tempfile.mkstemp(suffix=".mp3", prefix="tutor_answer_")[1])
    if tts_client is not None:
        try:
            with tts_client.audio.speech.with_streaming_response.create(
                model=model,
                voice=voice,
                input=text,
            ) as response:
                response.stream_to_file(audio_path)
            return audio_path
        except Exception as exc:
            message = str(exc).lower()
            is_405 = "405" in message or "method not allowed" in message
            if not is_405:
                raise
            print("⚠ Cloud TTS endpoint rejected streaming requests (405). Falling back to gTTS.")
    if gTTS is None:
        raise RuntimeError(
            "No TTS backend available. Install `gTTS` (`pip install gTTS`) or provide OPENAI_TTS_API_KEY/OPENAI_TTS_API_BASE for GPT-4o TTS.",
        )
    try:
        offline_tts = gTTS(text=text, lang="en")
        offline_tts.save(audio_path)
    except Exception as exc:
        raise RuntimeError(f"gTTS fallback failed: {exc}") from exc
    return audio_path


def play_audio_file(audio_path: Path) -> None:
    display(Audio(filename=str(audio_path)))


def speak_text(text: str) -> None:
    audio_path = synthesize_speech_to_file(text)
    play_audio_file(audio_path)
    try:
        audio_path.unlink(missing_ok=True)
    except Exception:
        pass

In [29]:
# Show the full path to the uploaded_pdfs folder
import os
pdf_folder_path = os.path.abspath(PDF_DIRECTORY)
print(f"\n📁 Your PDF folder is located at:")
print(f"   {pdf_folder_path}")
print(f"\nTo open it in File Explorer, run: explorer {pdf_folder_path}")


📁 Your PDF folder is located at:
   /content/uploaded_pdfs

To open it in File Explorer, run: explorer /content/uploaded_pdfs


## 4. PDF Processing Functions

In [30]:
def load_pdf(pdf_path):
    """Load and extract text from a PDF file."""
    print(f"Loading PDF: {pdf_path}")
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    print(f"✓ Loaded {len(documents)} pages")
    return documents

def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    """Split documents into smaller chunks for better retrieval."""
    print("Splitting documents into chunks...")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    chunks = text_splitter.split_documents(documents)
    print(f"✓ Created {len(chunks)} text chunks")
    return chunks

print("✓ PDF processing functions defined")

✓ PDF processing functions defined


## 5. Vector Store Setup

In [31]:
def create_vector_store(chunks, persist_directory=PERSIST_DIRECTORY):
    """Create a vector store from document chunks using local embeddings."""
    print("Creating embeddings and vector store...")

    # Use HuggingFace embeddings (free and local)
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cpu'}
    )

    # Create Chroma vector store
    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory=persist_directory
    )

    print(f"✓ Vector store created with {len(chunks)} chunks")
    return vectorstore

def load_vector_store(persist_directory=PERSIST_DIRECTORY):
    """Load existing vector store."""
    print("Loading existing vector store...")
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cpu'}
    )

    vectorstore = Chroma(
        persist_directory=persist_directory,
        embedding_function=embeddings
    )
    print("✓ Vector store loaded")
    return vectorstore

print("✓ Vector store functions defined")

✓ Vector store functions defined


## 6. Chatbot Tutor Class

In [32]:
class TutorChatbot:
    """A RAG-based chatbot tutor that answers questions based on uploaded documents."""

    def __init__(self, vectorstore, model_name=None, temperature=0.7):
        """Initialize the tutor chatbot."""
        self.vectorstore = vectorstore

        # Use the configured model name if not specified
        if model_name is None:
            model_name = MODEL_NAME

        # Create custom prompt template for tutoring
        self.prompt_template = """You are a helpful and patient tutor. Use the following context from the uploaded documents to answer the student's question.

If the answer is in the context, explain it clearly and in detail. If you're not sure or if the information isn't in the context, say so honestly and suggest related topics you can help with.

Be encouraging, clear, and educational in your responses. Break down complex concepts when needed.

Context from documents:
{context}

Previous conversation:
{chat_history}

Student's Question: {question}

Tutor's Answer:"""

        # Create the conversational chain
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True,
            output_key="answer"
        )

        self.llm = ChatOpenAI(
            model_name=model_name,
            temperature=temperature,
            openai_api_base="https://openrouter.ai/api/v1",
            default_headers={
                "HTTP-Referer": "https://github.com",
                "X-Title": "Tutor Chatbot"
            }
        )

        # Create retrieval chain
        self.qa_chain = ConversationalRetrievalChain.from_llm(
            llm=self.llm,
            retriever=self.vectorstore.as_retriever(search_kwargs={"k": 4}),
            memory=self.memory,
            return_source_documents=True,
            combine_docs_chain_kwargs={"prompt": PromptTemplate(
                template=self.prompt_template,
                input_variables=["context", "chat_history", "question"]
            )}
        )

        print("✓ Tutor chatbot initialized")

    def ask(self, question):
        """Ask a question to the tutor."""
        response = self.qa_chain({"question": question})
        return response["answer"], response["source_documents"]

    def reset_conversation(self):
        """Clear conversation history."""
        self.memory.clear()
        print("✓ Conversation history cleared")

print("✓ TutorChatbot class defined")

✓ TutorChatbot class defined


## 7. Upload Your PDF to Colab

Use the file upload widget below to upload your PDF directly to Colab.

In [ ]:
# Upload your PDF file to Colab
from google.colab import files
import shutil

print("📤 Click 'Choose Files' and select your PDF:")
uploaded = files.upload()

# Move uploaded file to the PDF directory
for filename in uploaded.keys():
    src = filename
    dst = os.path.join(PDF_DIRECTORY, filename)
    shutil.move(src, dst)
    print(f"\n✓ Uploaded: {filename}")
    print(f"  Saved to: {dst}")

    # Auto-set the PDF_FILENAME variable
    PDF_FILENAME = filename
    print(f"\n✓ PDF_FILENAME set to: '{PDF_FILENAME}'")

📤 Click 'Choose Files' and select your PDF:


In [ ]:
# Specify your PDF filename
PDF_FILENAME = "Drone Warfare.pdf"  # Replace with your actual PDF filename
pdf_path = os.path.join(PDF_DIRECTORY, PDF_FILENAME)

# Check if file exists
if not os.path.exists(pdf_path):
    print(f"⚠ PDF file not found: {pdf_path}")
    print(f"Please upload your PDF to the '{PDF_DIRECTORY}' folder and update the PDF_FILENAME variable.")
else:
    # Load and process the PDF
    documents = load_pdf(pdf_path)
    chunks = split_documents(documents)

    # Create vector store
    vectorstore = create_vector_store(chunks)

    print("\n" + "="*50)
    print("✓ PDF successfully processed and indexed!")
    print("="*50)

Loading PDF: ./uploaded_pdfs/Drone Warfare.pdf
✓ Loaded 10 pages
Splitting documents into chunks...
✓ Created 28 text chunks
Creating embeddings and vector store...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✓ Vector store created with 28 chunks

✓ PDF successfully processed and indexed!


## 8. Initialize the Tutor Chatbot

In [ ]:
# Initialize the tutor (make sure you've run the PDF processing cell first)
try:
    tutor = TutorChatbot(vectorstore)
    print("\n🎓 Tutor is ready! You can now ask questions about your document.")
except NameError:
    print("⚠ Please process a PDF first by running the previous cell.")

✓ Tutor chatbot initialized

🎓 Tutor is ready! You can now ask questions about your document.


## 9. Interactive Chat Interface

Ask questions to your tutor! The tutor will answer based on the uploaded PDF content.

In [ ]:
def chat_with_tutor():
    """Interactive chat session with the tutor."""
    print("=" * 70)
    print("🎓 TUTOR CHATBOT - Ask questions about your document")
    print("=" * 70)
    print("Type 'quit' or 'exit' to end the conversation")
    print("Type 'reset' to clear conversation history")
    print("Type 'sources' after a question to see source references")
    print("Type 'voice' to record a microphone question or 'voice-file <path>' to transcribe an audio file")
    print("Type 'tts on' to hear answers aloud and 'tts off' to mute them")
    print("=" * 70 + "\n")

    show_sources = False
    tts_enabled = False

    while True:
        user_input = input("\n📚 You: ").strip()

        if not user_input:
            continue

        command = user_input.lower()

        if command in ["quit", "exit"]:
            print("\n👋 Thank you for learning with me! Goodbye!")
            break

        if command == "reset":
            tutor.reset_conversation()
            continue

        if command == "sources":
            show_sources = True
            print("✓ Will show sources for the next answer")
            continue

        if command == "tts on":
            tts_enabled = True
            print("🔊 Text-to-speech enabled for future answers.")
            continue

        if command == "tts off":
            tts_enabled = False
            print("🔇 Text-to-speech disabled.")
            continue

        if command == "voice":
            try:
                user_input = capture_and_transcribe_question()
                if not user_input:
                    print("No speech detected. Try again.")
                    continue
                print(f"🎤 Transcribed question: {user_input}")
            except Exception as exc:
                print(f"Voice capture failed: {exc}")
                continue

        elif command.startswith("voice-file"):
            _, _, audio_path = user_input.partition(" ")
            if not audio_path.strip():
                print("Provide a path to the audio file after 'voice-file'.")
                continue
            try:
                user_input = transcribe_audio_file(Path(audio_path.strip()))
                if not user_input:
                    print("Transcription returned empty text. Try another file.")
                    continue
                print(f"🎤 Transcribed question: {user_input}")
            except Exception as exc:
                print(f"Voice transcription failed: {exc}")
                continue

        print("\n🎓 Tutor: ", end="")
        try:
            answer, sources = tutor.ask(user_input)
            print(answer)

            if tts_enabled:
                try:
                    speak_text(answer)
                except Exception as exc:
                    print(f"\n⚠ TTS failed: {exc}")

            if show_sources and sources:
                print("\n📖 Sources:")
                for i, doc in enumerate(sources, 1):
                    print(f"\n  [{i}] Page {doc.metadata.get('page', 'N/A')}:" )
                    print(f"      {doc.page_content[:200]}...")
                show_sources = False

        except Exception as e:
            print(f"Sorry, I encountered an error: {str(e)}")
chat_with_tutor()

🎓 TUTOR CHATBOT - Ask questions about your document
Type 'quit' or 'exit' to end the conversation
Type 'reset' to clear conversation history
Type 'sources' after a question to see source references


🎓 Tutor: Great question! The document provides a clear comparison between two types of drones: the Iranian-made **Shahed-136** and the Turkish **Bayraktar TB2**. Here’s a detailed breakdown of their differences:

### 1. **Type and Purpose**
- **Shahed-136**: This is a kamikaze drone, also known as a "loitering munition." It is designed for one-way missions, meaning it carries an explosive payload and is intended to crash into its target.
- **Bayraktar TB2**: This is a reusable unmanned aerial vehicle (UAV) designed for a variety of missions. It can carry multiple smart guided bombs or missiles, making it suitable for sustained operations.

### 2. **Size and Weight**
- **Shahed-136**: It is smaller, with a length of approximately **3.5 meters** and a wingspan of about **2.5 meters**. Its we

## 10. Example: Single Question Mode

Use this cell to ask individual questions without entering the interactive mode.

In [ ]:
# Ask a single question
question = "What are the main topics covered in this document?"

try:
    answer, sources = tutor.ask(question)

    print("📚 Question:", question)
    print("\n🎓 Answer:", answer)

    print("\n📖 Source References:")
    for i, doc in enumerate(sources, 1):
        page_num = doc.metadata.get('page', 'N/A')
        preview = doc.page_content[:150].replace('\n', ' ')
        print(f"\n[{i}] Page {page_num}: {preview}...")

except NameError:
    print("⚠ Please initialize the tutor first by running the previous cells.")

📚 Question: What are the main topics covered in this document?

🎓 Answer: The document covers several key topics related to drone warfare, particularly within the context of the Russia-Ukraine conflict. Here’s a breakdown of the main topics:

1. **Drone Attacks and Casualties**: The document details specific incidents of drone usage in the conflict, including a significant attack on Kyiv on April 24, 2025, where Russian forces conducted a combined missile and drone assault that resulted in at least 12 deaths and numerous injuries. This highlights the immediate impact of drone warfare on civilian populations.

2. **Ethical Responsibilities of Engineers**: It discusses the ethical dilemmas faced by engineers involved in the development of military technology, particularly drones. The document emphasizes the need for engineers to embed ethical safeguards in their designs, such as programming strict no-fire zones and maintaining human oversight in target selection processes. 

3. **Cost an

## 11. Utility Functions

In [ ]:
def add_new_pdf(pdf_filename):
    """Add a new PDF to the existing vector store."""
    pdf_path = os.path.join(PDF_DIRECTORY, pdf_filename)

    if not os.path.exists(pdf_path):
        print(f"⚠ PDF file not found: {pdf_path}")
        return False

    print(f"\n📄 Adding new PDF: {pdf_filename}")
    documents = load_pdf(pdf_path)
    chunks = split_documents(documents)

    # Add to existing vector store
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cpu'}
    )

    vectorstore.add_documents(chunks)
    print(f"✓ Added {len(chunks)} new chunks to the knowledge base")
    return True

def get_vector_store_stats():
    """Get statistics about the vector store."""
    try:
        collection = vectorstore._collection
        count = collection.count()
        print(f"📊 Vector Store Statistics:")
        print(f"   Total chunks indexed: {count}")
        print(f"   Storage location: {PERSIST_DIRECTORY}")
    except Exception as e:
        print(f"Could not retrieve stats: {str(e)}")

def clear_vector_store():
    """Clear the entire vector store (use with caution!)."""
    import shutil
    confirmation = input("⚠ This will delete all indexed documents. Type 'yes' to confirm: ")
    if confirmation.lower() == 'yes':
        shutil.rmtree(PERSIST_DIRECTORY)
        os.makedirs(PERSIST_DIRECTORY, exist_ok=True)
        print("✓ Vector store cleared")
    else:
        print("Operation cancelled")

print("✓ Utility functions defined")

✓ Utility functions defined


## 12. Usage Instructions

### How to Use This Tutor:

1. **Setup API Key**: Replace `OPENAI_API_KEY` in cell 3 with your actual OpenAI API key
   
2. **Upload PDF**: Place your PDF file in the `uploaded_pdfs` folder

3. **Process PDF**: Update the `PDF_FILENAME` variable in cell 7 and run the cell

4. **Initialize Tutor**: Run cell 8 to initialize the chatbot

5. **Start Chatting**:
   - Run cell 9 and uncomment `chat_with_tutor()` for interactive mode
   - Or use cell 10 to ask single questions

6. **(Optional) Enable Voice Interaction**:
   - Install a microphone-capable runtime with PortAudio support (local Python or Colab + system package)
   - If live recording is unavailable, use `voice-file <path>` to transcribe pre-recorded audio clips instead
   - Inside the chat loop, type `voice` to record a question or `voice-file <path>` to transcribe an existing audio clip
   - Type `tts on` or `tts off` to toggle spoken answers generated by GPT-4o TTS
   - For highest quality playback, export `OPENAI_TTS_API_KEY` (and optionally `OPENAI_TTS_API_BASE=https://api.openai.com/v1`). Without it, the notebook automatically falls back to the local `gTTS` helper after installation (`pip install gTTS`).

### Features:
- ✅ Local vector index (Chroma DB)
- ✅ Free embeddings (HuggingFace)
- ✅ Conversational memory
- ✅ Source citations
- ✅ Multiple PDF support
- ✅ Educational tutor personality
- ✅ Optional speech-to-text and text-to-speech controls

### Commands in Interactive Mode:
- `quit` or `exit` - End conversation
- `reset` - Clear conversation history
- `sources` - Show source references for next answer
- `voice` - Capture the next question from your microphone
- `voice-file <path>` - Transcribe an existing audio file
- `tts on` / `tts off` - Toggle spoken playback of answers